### Import libraries

In [ ]:
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, MinMaxScaler, OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 

from xgboost import XGBClassifier

# import own modules
sys.path.append("..")  # Adds higher directory to python modules path.
from scripts import features as ft
from scripts import preprocessing as pp
from scripts import evaluate_models as em

#plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle')

import pickle

---
## Preparations

### Load the CSV into a Dataframe

- load csv (or calculate again if not in data folder)
- update index=id
- drop useless columns
- find numerical & object columns

In [ ]:
# path to csv file
path_df = os.path.join("..", "data", "df_deepgaze2e.csv")

# get features - or recalculate
recalculate_df = False
if os.path.isfile(path_df) and not recalculate_df:
    df = pd.read_csv(path_df)
else:
    df = ft.get_features()
    df.to_csv(path_df, index=False)

# set id as index
df = df.set_index("id", drop=True)

# drop first batch of useless variables
df = df.drop(columns=['img', 'sp_idx'])
df = df.drop(columns=[col for col in df.columns if "_obj" in col])  # drop 'object' columns

# find numerical and categorical columns
num_cols = df.columns[df.dtypes != "object"]
cat_cols = df.columns[df.dtypes == "object"]

# print info
print(f" -> dataframe has {df.shape[0]} instances and {df.shape[1]} columns")
print(f" -> there are {len(num_cols)} numerical columns")
print(f" -> there are {len(cat_cols)} categoricals columns")

### Checking for highly correlated columns
think after running this lines, which column to additionally drop

In [ ]:
# Computing the correlation matrix
corr_matrix = df[num_cols].corr()

# Find pairs with correlation >= 0.8
high_corr_pairs = np.column_stack(
    np.where((np.abs(corr_matrix) >= 0.8) & (corr_matrix != 1))
)
high_corr_cols = []

# Extracting and printing the pairs
seen_pairs = set()
for i, j in high_corr_pairs:
    col1, col2 = corr_matrix.columns[i], corr_matrix.columns[j]
    if (col2, col1) not in seen_pairs:
        print(
            f"Correlation between {col1} and {col2} is {round(corr_matrix.iloc[i, j], 3)}"
        )
        seen_pairs.add((col1, col2))
        seen_pairs.add((col2, col1))
        high_corr_cols.append(col1)
        high_corr_cols.append(col2)

### further processing

... drop more columns, create new ones, handle highly correlating columns.....

### Split into train & test sets

As soon as the dataset is in its final form, perform train-test-split with our own split function to have out 30-image-set always as our test set.

In [ ]:
# prepare features and target
X = df
y = X.pop("asd")
num_cols = X.columns[X.dtypes != "object"]
cat_cols = X.columns[X.dtypes == "object"]

# train-test-split
X_train, X_test, y_train, y_test = pp.split(X, y)

# print info
print(f"train-set has '{len(y_train)}' samples & '{X.shape[1]}' features")
print(f"test-set has '{len(y_test)}' samples - out of '{df.shape[0]}'")
print(f"  ~ {len(y_test) / df.shape[0] * 100:.2f}% of full dataset")

### Set variables

- define `metric`
- behavior for saving models as pickles
- defaults for model-objects

In [ ]:
# metric
ftwo_scorer = make_scorer(fbeta_score, beta=2)

# defaults
RSEED = 42
cv = 10
n_jobs = -1
verbose = 1

---
## Model - Preparations

### Column Transformers for Data Preproccessing
these are only used for certain Models, which we expect to perform better with Preproccessed Data

In [ ]:
# add other transformations at the end if needed
transformer = [("scaler", MinMaxScaler(), num_cols),
               ("ohe", OneHotEncoder(drop="first"), cat_cols  )]
               
preprocessing = ColumnTransformer(transformer,
                                  remainder="passthrough")

###  Pipelines for each Model

In [ ]:
# Random Forest: no scaling / no encoding
rf_pipeline = Pipeline([
    ("classifier", RandomForestClassifier())
])

# XGBoost: apply scaling / encoding
xgb_pipeline = Pipeline([
    ("preprocessor", preprocessing),
    ("classifier", XGBClassifier())
])

# Support Vector Classifier: apply scaling / encoding
svc_pipeline = Pipeline([
    ("preprocessor", preprocessing),
    ("classifier",SVC())
])

# Logistic Regression: apply scaling / encoding
log_pipeline = Pipeline([
    ("preprocessor", preprocessing),
    ("classifier", LogisticRegression(max_iter=1000))
])

### Parameter Grids

In [ ]:
# Random Forest
param_grid_rf = {
    "classifier__n_estimators": [100, 200, 300],
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 5, 10],
    "classifier__min_samples_leaf": [1, 2, 4]
}

# XGBoost
param_grid_xgb = {
    "classifier__n_estimators": [100, 200, 300],
    "classifier__max_depth": [3, 5, 7, None],
    "classifier__learning_rate": [0.1, 0.01, 0.001]
}

# Support Vector Classifier
param_grid_svc = {
    'classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Kernel types to try
    'classifier__C': [0.1, 1, 10, 100],  # Regularization parameter values
    'classifier__gamma': ['scale', 'auto'],  # Gamma parameter for RBF kernel
    'classifier__degree': [2, 3, 4]  # Degree of the polynomial kernel (only for poly kernel)
}

# Logistic Regression
param_grid_logreg = {
    'classifier__penalty': ['l1', 'l2'],  # Penalty type: l1 (Lasso) or l2 (Ridge)
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]  # Regularization strength
}

---
## Modeling

### XGBoost

In [ ]:
y_train=np.array(y_train)

In [ ]:
y_train

In [ ]:
X_train

In [ ]:
# Create GridSearchCV object
grid_search_xgb = GridSearchCV(
    xgb_pipeline,
    param_grid=param_grid_xgb,
    cv=cv,
    scoring=ftwo_scorer,
    n_jobs=n_jobs,
    verbose=verbose,
    error_score='raise'
)
grid_search_xgb.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params_xgb = grid_search_xgb.best_params_
best_est_xgb = grid_search_xgb.best_estimator_
print("Best params for XGB are:", best_params_xgb)
print("Best est for XGB are:", best_est_xgb)

In [ ]:
# predict & proba
pred_test = grid_search_xgb.predict(X_test)
proba_test = grid_search_xgb.predict_proba(X_test)

pred_train = grid_search_xgb.predict(X_train)
proba_train = grid_search_xgb.predict_proba(X_train)

In [ ]:
# evaluate model
em.report(
    y_train=y_train,
    y_train_pred=pred_train,
    y_train_proba=proba_train,
    y_test=y_test,
    y_test_pred=pred_test,
    y_test_proba=proba_test,
)

In [ ]:
# learning curves - for one model - f2 score
em.learning(best_est_xgb, X_train, y_train)

In [ ]:
em.model_info(best_est_xgb)

1. Feature importance using permutation importance
* performs better, when model in overfitting
* Train a  model and record the score on the validation set. Re-shuffle values for one feature, use the model to predict again, and calculate scores on the validation set. The feature importance for the feature is the difference between the baseline in 1 and the permutation score in 2. Repeat the process for all features.

In [ ]:
from sklearn.inspection import permutation_importance
#calculate permutation importance for test data 
result_test = permutation_importance(
    grid_search_xgb, X_test, y_test, n_repeats=20, random_state=42, n_jobs=2
)

sorted_importances_idx_test = result_test.importances_mean.argsort()
importances_test = pd.DataFrame(
    result_test.importances[sorted_importances_idx_test].T,
    columns=X.columns[sorted_importances_idx_test],
)

#calculate permutation importance for training data 
result_train = permutation_importance(
    grid_search_xgb, X_train, y_train, n_repeats=20, random_state=42, n_jobs=2
)

sorted_importances_idx_train = result_train.importances_mean.argsort()
importances_train = pd.DataFrame(
    result_train.importances[sorted_importances_idx_train].T,
    columns=X.columns[sorted_importances_idx_train],
)

f, axs = plt.subplots(1,2,figsize=(15,5))

importances_test.plot.box(vert=False, whis=10, ax = axs[0])
axs[0].set_title("Permutation Importances (test set)")
axs[0].axvline(x=0, color="k", linestyle="--")
axs[0].set_xlabel("Decrease in accuracy score")
axs[0].figure.tight_layout()

importances_train.plot.box(vert=False, whis=10, ax = axs[1])
axs[1].set_title("Permutation Importances (train set)")
axs[1].axvline(x=0, color="k", linestyle="--")
axs[1].set_xlabel("Decrease in accuracy score")
axs[1].figure.tight_layout()

X axis shows decrease in accuracy score, when feature is shuffled. --> features with higher negative values are more important for performance of model. Features with positive values: not so important.

Difference between training and test data 

### Selecting features:
sal_weighted_duration_sum, sp_fix_duration_ms_total, sp_fix_count, sal_max, sal_first_above_0.9*max_rank, sal_sum, sp_distance_to_centre_px_mean, obj_t_rel_on_inanimate, obj_n_fix_animate, object_n_fix_face, obj-n_fix_inanimate

In [ ]:
# Run XGBoost with selected features 
features_to_remove = ['sal_weighted_duration_sum', 'sp_fix_duration_ms_total', 'sp_fix_count', 
                      'sal_max', 'sal_first_above_0.9*max_rank', 'sal_sum', 
                      'sp_distance_to_centre_px_mean', 'obj_t_rel_on_inanimate', 
                      'obj_n_fix_animate', 'obj_n_fix_face', 'obj_n_fix_inanimate']

# Drop the specified features
df2 = df.drop(features_to_remove, axis=1)

In [ ]:
# prepare features and target
X2 = df2
num_cols = X2.columns[X2.dtypes != "object"]
cat_cols = X2.columns[X2.dtypes == "object"]

# train-test-split
X_train2, X_test2, y_train, y_test = pp.split(X2, y)

In [ ]:
# Create GridSearchCV object
grid_search_xgb2 = GridSearchCV(
    xgb_pipeline,
    param_grid=param_grid_xgb,
    cv=cv,
    scoring=ftwo_scorer,
    n_jobs=n_jobs,
    verbose=verbose,
    error_score='raise'
)
grid_search_xgb2.fit(X_train2, y_train)

# Get the best parameters and best estimator
best_params_xgb2 = grid_search_xgb2.best_params_
best_est_xgb2 = grid_search_xgb2.best_estimator_
print("Best params for XGB are:", best_params_xgb2)
print("Best est for XGB are:", best_est_xgb2)

In [ ]:
xgb_opt =(
    xgb_pipeline,
    param_grid=best_params_xgb,
    cv=cv,
    scoring=ftwo_scorer,
    n_jobs=n_jobs,
    verbose=verbose,
    error_score='raise'
)
grid_search_xgb.fit(X_train, y_train)

Best parameters: 
Best params for XGB are: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__n_estimators': 100}
Best est for XGB are: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', MinMaxScaler(),
                                                  Index(['sp_fix_count', 'sp_fix_duration_ms_total', 'sp_fix_duration_ms_mean',
       'sp_fix_duration_ms_var', 'sp_len_px_total',
       'sp_saccade_amplitude_px_mean', 'sp_saccade_amplitude_px_var',
       'sp_distance_to_centre_px_mean', 'sp_distance_to_centre_px_...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=100, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

# Converting data into DMatrix format (optimized data structure for XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Setting parameters for XGBoost model
params = {
    'objective': 'binary:logistic',  # for binary classification
    'eval_metric': 'logloss',         # evaluation metric
    'max_depth': 3                     # maximum depth of trees
}


Next, we convert the data into the DMatrix format, which is the optimized data structure for XGBoost.
We define parameters for the XGBoost model, such as the objective function, evaluation metric, and maximum depth of trees.
